In [ ]:
# Import packages and other functions
import os
import re
import time
import random
import requests
import numpy as np
import pandas as pd
from datetime import datetime
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn import preprocessing 


In [ ]:
def clean_apartment_data(df):
    """Clean up the apartments raw data."""
    # Start by dropping any duplicate entries.
    df = df.drop_duplicates()
    
    # Square footage comes as "[0-9] - [0-9] sq ft"
    # 1. Strip commas and " sq ft", then split the remaining into three columns, titled "min_square_footage", "hyphen", and "max_square_footage."
    # 2. Drop the hyphen column.
    # 3. If the max column is null, fill it with whatever's in the min column.
    df[['min_square_footage', 'hyphen', 'max_square_footage']] = df['square_footage'].str.replace(' sq ft', '').str.replace(',', '').str.split(expand=True)
    df = df.drop(['square_footage', 'hyphen'], 1)
    df['max_square_footage'][df['max_square_footage'].isnull()] = df['min_square_footage'].loc[df['max_square_footage'].isnull()]

    # Repeat the above process with the price column
    df['price'] = df['price'].str.replace('Call for Rent', '')
    df[['min_price', 'hyphen', 'max_price']] = df['price'].str.split(expand=True)
    df = df.drop(['price', 'hyphen'], 1)
    df['max_price'][df['max_price'].isnull()] = df['min_price'].loc[df['max_price'].isnull()]

    # Strip nonnumeric characters
    for column in ['beds', 'baths', 'min_price', 'max_price']:
        df[column] = df[column].str.replace(r'[A-Za-z ,$]*', '')

    # Convert data to numeric
    for column in ['beds', 'baths', 'min_price', 'max_price', 'max_square_footage', 'min_square_footage']:
        df[column] = pd.to_numeric(df[column])

    # Remove studio apartments
    df = df[~df['beds'].isnull()]

    # Fill in missing availability data
    df['availability'][df['availability'].isnull()] = 'No availability provided'

    # Price per square foot
    df['max_ppf'] = df['max_price'] / df['max_square_footage']
    df['min_ppf'] = df['min_price'] / df['min_square_footage']

    return(df)


def get_commute_lengths(origins, destination, api_key, sleep_time=0.5):
    """
    Hit the google Distance Matrix api for the distances between origins and destinations.
    More specifically, take a list of origins and a list of destinations and return an array of json objects.
    We included a half a second sleep time by default to avoid from overwhelming the page.
    """
    # Empty array to store json distance objects
    distance_objects = []

    # This is the url to access the maps api, we need to append the origin and destination as well as api key
    distance_matrix_url = 'https://maps.googleapis.com/maps/api/distancematrix/json?'

    # For each origin in our set of origins and for each destination in our set of destinations:
    #    search for the distance between that and each destination.
    for origin in origins:
        time.sleep(sleep_time)
        print(f"Searching for the information about the drive from {origin} to {destination}.")
        search_result = requests.get(distance_matrix_url + 'origins=' + origin + '&destinations=' + destination + '&key=' + api_key)
        distance_objects.append(search_result.json())
    
    return(distance_objects)

In [ ]:
###############
# Import Data #
###############
# Since we date the files we save, open the last file in the directory.
directory_location = 'data/scraped_data/'
newest_file = os.listdir(directory_location)[-1]
df_0 = pd.read_csv(directory_location + newest_file)
print("Retrieving information from", newest_file)

##############
# Clean Data #
##############
df_cleaned = clean_apartment_data(df_0)
df_cleaned


In [ ]:
##########################
# Append Commute Lengths #
##########################

# 
# 
# 
# 
destination = ['210 Carnegie Center, Princeton, NJ']
origins = df_cleaned['residence_address'].unique()
commute_json_objects = get_commute_lengths(origins=origins, destination=destination, api_key= #insert api key here)
distances = [d['rows'][0]['elements'][0]['duration']['value'] for d in commute_json_objects]
commute_durations = pd.DataFrame({'residence_address' : origins, 'commute_in_seconds' : distances})
# commute_durations.to_csv('data/commute_length.csv', index=False) # uncommment to save a csv of the distance set
df = df_cleaned.merge(commute_durations, on=['residence_address'], how='left')
df['commute_in_minutes'] = df['commute_in_seconds'] / 60
df

In [ ]:
""" Scale the data so we can use come up with the 'apartment rating' """

# Scaler
min_max_scaler = preprocessing.MinMaxScaler()

# Invert these to get the apartment rating components going in the same direction
for column in [
    'min_price', 'max_price', 
    # 'commute_in_minutes'
    ]:
    df[column + '_inv'] = 1 / df[column]

scaled_data = pd.DataFrame(min_max_scaler.fit_transform(df[['min_square_footage', 'max_square_footage', 'min_price', 'max_price', 'min_ppf', 'max_ppf',
                                                            # 'commute_in_minutes', 
                                                            'min_price_inv', 'max_price_inv', 
                                                            # 'commute_in_minutes_inv'
                                                            ]]), 
                            columns=['scaled_' + c for c in ['min_square_footage', 'max_square_footage', 'min_price', 'max_price', 'min_ppf', 'max_ppf',
                                    #  'commute_in_minutes', 
                                     'min_price_inv', 'max_price_inv',
                                    #   'commute_in_minutes_inv'
                                      ]])
df = df.join(scaled_data)
df['apartment_rating'] = (((df['scaled_min_square_footage']) ** 2) + ((df['scaled_min_price_inv']) ** 2) + ((df['scaled_commute_in_minutes_inv']) ** 2)) ** (1/2)


In [ ]:
"""Apartment features should be filled in as you look at the residences. I haven't looked into automating that yet."""

# Does the building have a pool?
df['pool'] = 'unknown'
df.loc[df['residence_name'].isin([
                                    # insert residence names here
                                    ]), ['pool']] = 'y'
df.loc[df['residence_name'].isin([
                                    # insert residence names here
                                    ]), ['pool']] = 'n'

df['patio'] = 'unknown'
df.loc[df['residence_name'].isin([
                                  # insert residence names here
                                    ]), ['patio']] = 'y'
df.loc[df['residence_name'].isin([
                                  # insert residence names here
                                    ]), ['patio']] = 'n'

# df.loc[[302], ['patio']] = 'n'
df['laundry'] = 'unknown'
df.loc[df['residence_name'].isin([
                                # insert residence names here
                                    ]), ['laundry']] = 'communal'
df.loc[df['residence_name'].isin([
                                    # insert residence names here
                                    ]), ['laundry']] = 'in unit'
df.loc[df['residence_name'].isin(['Cedar Manor', 'Pike Run Meadows', 'Summerfields Lofts', 'The Grove Somerset', 'Merrieworld', 'Plaza Square Apartment Homes', 'The Aspire']
), ['dishwasher']] = 'y'

# Slice the data frame to get specific apartments and put them in our ploting data frame
pf = df[
        # insert filters here
        ]
        
# 

In [ ]:
# 2D interactive chart
# Add whichever columns to the hover data to see it on mouseover
fig = px.scatter(pf, 
                    x = 'min_square_footage',
                    # x='apartment_rating', 
                    # y = 'scaled_min_price',
                    y = 'min_price',
                    color='residence_name', 
                    # size='commute_in_minutes',
                    # text = 'model_name',
                    # title = 'Apartments: Square Footage by Price',
                    # labels={'min_price' : 'Price', 'min_square_footage' : 'Square Footage'},
                    hover_data=['model_name', 'min_ppf'],
                    )
# fig.update_traces(textposition='top center')

fig.show()
# fig.write_html('apartments_2d.html') # Uncomment this to save it as it's own interactive chart  

In [ ]:
# Same as the above, but 3D
fig_3d = px.scatter_3d(pf,
                    x="min_square_footage", 
                    y="min_price", 
                    z='commute_in_minutes',
                    color='search_location', 
                    hover_data=['min_ppf', 'model_name']
                    # hover_data=['residence_name', 'model_name', 'min_ppf', 'commute_in_minutes', 'availability', 'laundry', 'dishwasher', 'pool', 'patio', pf.index]
                    )
fig_3d.show()
# fig_3d.write_html('apartments_3d.html')